In [82]:
from __future__ import annotations

import json
import os
from pathlib import Path
from queue import Queue

import numpy as np
import pandas as pd
from librosa import load, feature
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

PATH_TO_FILES = Path('files/')

In [39]:
class FileFinder:
    suffixes: set[str]
    _queue: Queue[Path]

    def __init__(self, root, suffixes):
        self.root = Path(root)
        self._queue = Queue()
        self.suffixes = suffixes

        self._queue.put(self.root)

    def get_next_file(self):
        if self._queue.empty():
            print("Queue is empty")
            return None

        res = self._queue.get()
        if res.is_dir():
            for child in res.iterdir():
                if child.suffix[1:].lower() in self.suffixes or child.is_dir():
                    # print(f'Added {child.name} to queue [{"folder" if child.is_dir() else child.suffix}]')
                    self._queue.put(child)
            return Ellipsis
        else:
            return res if res.suffix[1:] in self.suffixes else None

    def reset(self):
        self._queue = Queue()
        self._queue.put(self.root)

    def get_rel_path(self, file: Path):
        return Path(os.path.relpath(file.parent, self.root).__str__())

    def get_full_path(self, file: Path):
        return self.root.joinpath(file)

class FileIO:
    _data: list[dict, ...]

    def __init__(self, path):
        self.path = Path(path)
        self._data = ...
    
    def get_data(self):
        with open(self.path) as json_file:
            self._data = json.load(json_file)
        return self._data
    
    def set_data(self, data):
        if type(data) is list:
            self._data = data
            with open(self.path, 'w') as json_file:
                json_file.write(json.dumps(self._data, sort_keys=False, indent=4))
        

In [91]:
class FeatureExtractor:
    def __init__(self):
        self.mfccs = 10
        self.rate = 22050
    

    def mfcc(self, audio):
        return np.mean(feature.mfcc(y=audio, sr=self.rate, n_mfcc=self.mfccs).tolist(), axis=1)
    
    def contrast(self, audio):
        return np.mean(feature.spectral_contrast(y=audio, sr=self.rate).tolist())
        

In [92]:
finder = FileFinder(PATH_TO_FILES, ('au', ))
output = FileIO('data.json')
extractor = FeatureExtractor()
data = []

while True:
    curr = finder.get_next_file()
    if not curr:
        break
    elif curr is ...:
        continue
    else:
        curr_file, fs = load(curr, duration=7)
        data.append(dict(
            id=curr.name,
            genre=curr.parent.name,
            **dict(zip((f'mfcc{n}' for n in range(extractor.mfccs)), extractor.mfcc(curr_file))),
            contrast=extractor.contrast(curr_file),
            prediction=''
        ))
        # print(curr.parent.name, curr.name.split('.')[1])

output.set_data(data)
print('Success')

Queue is empty
Success


In [93]:
features = [*(f'mfcc{n}' for n in range(extractor.mfccs)), 'contrast']

frame = pd.DataFrame(data)
categories = frame[['genre']]
predictors = frame[features]
predictors.columns = features

pre_train, pre_test, cat_train, cat_test = train_test_split(predictors, categories, test_size=0.33, random_state=5)
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200000)
model.fit(pre_train, cat_train.genre)
predictions = model.predict(pre_test)
# confusion_matrix(cat_test, predictions)
print(classification_report(cat_test, predictions))

              precision    recall  f1-score   support

   classical       0.82      0.82      0.82        11
      hiphop       0.57      0.40      0.47        20
        jazz       0.58      0.94      0.71        16
         pop       0.59      0.81      0.68        16
      reggae       0.70      0.35      0.47        20

    accuracy                           0.63        83
   macro avg       0.65      0.66      0.63        83
weighted avg       0.64      0.63      0.60        83

